In [1]:
import sys
sys.path.append('../')

In [2]:
"""
# Description:  Outputs a Microsoft Excel sheet to be analyzed with a pivot table to 
#               produce a footprint acres report as well as a point featureclass.
#
#               "Footprint" is defined as treated acres within a treatment area 
#               irrespective of how many activities occurred within the treatment area.
#
#               This process uses a "Spaghetti and Meatballs" approach.  We first 
#               create the Spaghetti by using the "Feature to Polygon" tool.  
#               We use the "Identify" to attach ownership, vegetation, and region 
#               attributes to the Spaghetti. Then we create Meatballs using the 
#               "Feature to Points" tool.  Finally, we create the report with 
#               "Summarize Within" to find the Maximum Activity Quantity within a polygon.
#
# Known Issues: This tool under reports CalTrans footprints due to the way CalTrans 
#               reports treatment activities by road segment.
#
# Author: Spatial Informatics Group LLC
# Version: 1.0.0
# Date Created: Jan 24, 2024
"""

import os
import datetime
import arcpy
from scripts.utils import init_gdb, check_schema_lock, delete_scratch_files, og_file_input

workspace, scratch_workspace = init_gdb()

arcpy.EnvManager(
    workspace=workspace,
    scratchWorkspace=scratch_workspace,
    outputCoordinateSystem=arcpy.SpatialReference("NAD 1983 California (Teale) Albers (Meters)"),  # WKID 3310
    cartographicCoordinateSystem=arcpy.SpatialReference("NAD 1983 California (Teale) Albers (Meters)"),  # WKID 3310
    extent="xmin=-374900, ymin=-604500, xmax=540100, ymax=450000, spatial_reference='NAD 1983 California (Teale) Albers (Meters)'",
    preserveGlobalIds=True,
    qualifiedFieldNames=False,
    transferDomains=False,
    transferGDBAttributeProperties=False,
    overwriteOutput=True,
)

date_id = datetime.datetime.now().strftime("%Y-%m-%d").replace("-", "")  # like 20221216

year = str(2023)

# INPUTS
# change path to the most updated inputs
treat_poly = og_file_input(prefix="Treat_n_harvests_polygons"+year,filetype="Polygon",gdb=os.path.join(workspace, "d_Appended"),)
input_fc_poly = os.path.join(workspace, "d_Appended", treat_poly)
treat_pt = og_file_input(prefix="Treat_n_harvests_points"+year,filetype="Point",gdb=os.path.join(workspace, "d_Appended"),)
input_fc_pt = os.path.join(workspace, "d_Appended", treat_pt)
treat_ln = og_file_input(prefix="Treat_n_harvests_lines"+year,filetype="Line",gdb=os.path.join(workspace, "d_Appended"),)
input_fc_ln = os.path.join(workspace, "d_Appended", treat_ln)

WFR_TF_Template = os.path.join(workspace, "a_Reference", "WFR_TF_Template")
Reference_Data = os.path.join(workspace, "a_Reference", "Own_Veg_Region_WUI")

# Edit where clause for footprints report
# And (Agency = 'Industrial Timber' or AGENCY = 'CALEPA' Or AGENCY = 'CALSTA' Or AGENCY = 'CNRA' Or AGENCY = 'DOD' Or AGENCY = 'DOI' Or AGENCY = 'USDA')"

# OUTPUTS
output_footprint = os.path.join(workspace, "f_Report_Data", f"Footprint_Report_{date_id}")
output_footprint_pts = os.path.join(workspace, "f_Report_Data", f"Footprint_pts_{date_id}")
#output_excel_report = os.path.join("..", f"Draft_Footprints_Report_Counts_to_MAS_{year}_{date_id}a.xlsx")

data = arcpy.GetParameterAsText(0)

check_schema_lock(input_fc_poly)
check_schema_lock(input_fc_ln)
check_schema_lock(input_fc_pt)
check_schema_lock(output_footprint)

Available for schema lock: D:\WORK\wildfire\Interagency-Tracking-System\its\Interagency Tracking System.gdb\d_Appended\Treat_n_harvests_polygons2023_20240927
Available for schema lock: D:\WORK\wildfire\Interagency-Tracking-System\its\Interagency Tracking System.gdb\d_Appended\Treat_n_harvests_lines2023_20240927
Available for schema lock: D:\WORK\wildfire\Interagency-Tracking-System\its\Interagency Tracking System.gdb\d_Appended\Treat_n_harvests_points2023_20240927
Available for schema lock: D:\WORK\wildfire\Interagency-Tracking-System\its\Interagency Tracking System.gdb\f_Report_Data\Footprint_Report_20240927


In [3]:
arcpy.env.overwriteOutput = True

In [4]:
def update_pt(input_fc_pt):
    Updated_Input_Table = arcpy.management.AddField(
        in_table=input_fc_pt, 
        field_name="BufferMeters", 
        field_type="DOUBLE"
    )
    
    select_1 = arcpy.management.SelectLayerByAttribute(
        in_layer_or_view=Updated_Input_Table,
        selection_type="NEW_SELECTION",
        where_clause="ACTIVITY_QUANTITY IS NOT NULL And ACTIVITY_UOM = 'AC'",
    )
    
    calc_buffer_1 = arcpy.management.CalculateField(
        select_1,
        "BufferMeters",
        "math.sqrt((!ACTIVITY_QUANTITY!*4046.86)/3.14159)",
        expression_type="PYTHON3",
    )
    
    select_2 = arcpy.management.SelectLayerByAttribute(
        in_layer_or_view=calc_buffer_1,
        selection_type="SUBSET_SELECTION",
        where_clause="COUNTS_TO_MAS = 'YES' And BufferMeters IS NOT NULL",
    )
    
    scratch_pt_3 = os.path.join(scratch_workspace, "scratch_pt_3")
    
    buffer_pts_1 = arcpy.analysis.PairwiseBuffer(
        in_features=select_2,
        out_feature_class=scratch_pt_3,
        buffer_distance_or_field="BufferMeters",
    )
    return buffer_pts_1

In [5]:
def update_ln(input_fc_ln):
    
    # LINES
    # Create polygons from lines based on Activity Acres

    add_field_2 = arcpy.management.AddField(
        in_table=input_fc_ln, 
        field_name="BufferMeters", 
        field_type="DOUBLE"
    )
    
    scratch_ln_1 = os.path.join(scratch_workspace, "Treat_n_harvests_lns_CopyFeatures")
    
    select_3 = arcpy.Select_analysis(
        in_features=add_field_2,
        out_feature_class=scratch_ln_1,
        where_clause="ACTIVITY_QUANTITY IS NOT NULL And ACTIVITY_UOM = 'AC' And Shape_Length IS NOT NULL And Shape_Length <> 0",
    )
    
    
    Treat_n_harvests_points_20221030_3_ = arcpy.management.CalculateField(
        in_table=select_3,
        field="BufferMeters",
        expression="(!ACTIVITY_QUANTITY!*4046.86)/!Shape_Length!/2",
        expression_type="PYTHON3",
    )
    
    scratch_ln_2 = os.path.join(scratch_workspace, "Treat_n_harvests_lns_CopyFeatures_Yes")
    
    select_4 = arcpy.Select_analysis(
        in_features=add_field_2,
        out_feature_class=scratch_ln_2,
        where_clause="COUNTS_TO_MAS = 'YES'",
    )
    
    select_5 = arcpy.management.SelectLayerByAttribute(
        in_layer_or_view=select_4,
        selection_type="NEW_SELECTION",
        where_clause="BufferMeters >= 200 And Source = 'CalTrans'",
        invert_where_clause="INVERT",
    )
    
    scratch_ln_3 = os.path.join(scratch_workspace, "Treat_n_harvests_lns_Buffer")
    arcpy.analysis.PairwiseBuffer(
        in_features=select_5,
        out_feature_class=scratch_ln_3,
        buffer_distance_or_field="BufferMeters",
    )  # line_end_type="FLAT")

    return scratch_ln_3

In [6]:
def update_poly(input_fc_poly):
    # POLYGON
    
    select_6 = arcpy.management.SelectLayerByAttribute(
        in_layer_or_view=input_fc_poly, 
        where_clause="COUNTS_TO_MAS = 'YES'"
    )
    
    select_7 = arcpy.management.SelectLayerByAttribute(
        in_layer_or_view=select_6,
        selection_type="SUBSET_SELECTION",
        where_clause="TREATMENT_AREA < 100000",
    )
    
    scratch_poly_1 = os.path.join(scratch_workspace, "Treat_n_harvests_poly_CopyFeatures")
    
    arcpy.management.CopyFeatures(
        in_features=select_7,
        out_feature_class=scratch_poly_1,
    )

    return scratch_poly_1

In [7]:

def get_footprint(input_append, output_footprint, output_footprint_pts, year_start=2020, year_end=2023):

    footprint_lst = []
    footprint_pt_lst = []


    for year in range(year_start, year_end+1):
        year = str(year)
        print("Processing year: ", year)
    
        # This Feature Class is used to create both the Spaghetti and the Meatballs
        
        Footprints_where_clause = "(Year = {})".format(year) 
        
        select_8 = os.path.join(scratch_workspace, "Footprints_Append_Where")
        
        arcpy.analysis.Select(
            in_features=input_append,
            out_feature_class=select_8,
            where_clause=Footprints_where_clause,
        )
        
        # by pass year due to logistic change to include all year data
        
        # check results
        result = arcpy.management.GetCount(select_8)
        print("{} has {} records".format(select_8, result[0]))

        #
        # Create Meatballs
        #
        
        Meatballs = os.path.join(workspace, "Meatballs"+year)
        
        arcpy.management.FeatureToPoint(
            in_features=select_8,
            out_feature_class=Meatballs,
            point_location="INSIDE",
        )
        
        arcpy.DefineProjection_management(
            Meatballs, arcpy.SpatialReference("NAD 1983 California (Teale) Albers (Meters)")
        )


        #
        # Create the Spaghetti
        #
        
        Spaghetti = os.path.join(scratch_workspace, "Spaghetti"+year)
        arcpy.analysis.PairwiseDissolve(
            in_features=select_8,
            out_feature_class=Spaghetti,
            dissolve_field=["TRMTID_USER"],
        )
        
        Spaghetti_FeatureToPolygon = os.path.join(scratch_workspace, "Spaghetti_FeatureToPolygon")
        
        arcpy.management.FeatureToPolygon(
            in_features=[Spaghetti], 
            out_feature_class=Spaghetti_FeatureToPolygon
        )
    
        # check results
        result = arcpy.management.GetCount(Spaghetti_FeatureToPolygon)
        print("{} has {} records".format(Spaghetti_FeatureToPolygon, result[0]))
        
        Spaghetti_Delete_Fields = arcpy.management.DeleteField(
            in_table=Spaghetti_FeatureToPolygon,
            drop_field=["FID_Spaghetti_Dissolve", "TRMTID_USER"],
        )
    
    
        # Add Sauce: Ownership, Vegetation, and Region attributes to the Spaghetti
        Sauce = Reference_Data
        
        Spaghetti_n_Sauce = os.path.join(workspace, "Spaghetti_n_Sauce"+year)
        
        arcpy.analysis.Identity(
            in_features=Spaghetti_Delete_Fields,
            identity_features=Sauce,
            out_feature_class=Spaghetti_n_Sauce,
            join_attributes="NO_FID",
        )
        
        arcpy.DefineProjection_management(
            Spaghetti_n_Sauce,
            arcpy.SpatialReference("NAD 1983 California (Teale) Albers (Meters)"),
        )
    
    
        # Ensure the ownership for CalTrans projects are State
        
        CalTrans_Projects = os.path.join(scratch_workspace, "CalTrans_Projects")
        
        arcpy.analysis.Select(
            in_features=Footprints_Append,
            out_feature_class=CalTrans_Projects,
            where_clause="AGENCY = 'CALSTA'",
        )
        
        dissolve_1 = os.path.join(scratch_workspace, "CalTrans_Projects_Dissolve")
        
        arcpy.analysis.PairwiseDissolve(
            in_features=CalTrans_Projects,
            out_feature_class=dissolve_1,
            dissolve_field=["AGENCY"],
        )
    
    
        # Select CalTrans Projects to set Ownership to State
        select_9 = arcpy.management.SelectLayerByLocation(
            in_layer=Spaghetti_n_Sauce,
            overlap_type="HAVE_THEIR_CENTER_IN",
            select_features=dissolve_1,
            search_distance=10,
            selection_type="NEW_SELECTION",
        )
        
        calc_field_1 = arcpy.management.CalculateField(
            in_table=select_9,
            field="PRIMARY_OWNERSHIP_GROUP",
            expression='"STATE"',
        )
        
        select_10 = arcpy.management.SelectLayerByAttribute(
            in_layer_or_view=calc_field_1, 
            selection_type="CLEAR_SELECTION"
        )
    
    
        add_field_3 = arcpy.management.AddField(
            in_table=select_10,
            field_name="FootprintAcres",
            field_type="DOUBLE",
            field_precision=8,
            field_scale=1,
        )
        
        calc_geom_1 = arcpy.management.CalculateGeometryAttributes(
            in_features=add_field_3,
            geometry_property=[["FootprintAcres", "AREA"]],
            area_unit="ACRES_US",
        )
    
        
        #
        # Make Dinner
        #
        
        Dinner = os.path.join(scratch_workspace, "Dinner")
        
        make_dinner = arcpy.SummarizeWithin_analysis(
            in_polygons=Spaghetti_n_Sauce,
            in_sum_features=Meatballs,
            out_feature_class=Dinner,
            keep_all_polygons="ONLY_INTERSECTING",
            sum_fields=[
                # ["ACTIVITY_QUANTITY", "Sum"],
                ["ACTIVITY_QUANTITY", "Mean"],
                # ["ACTIVITY_QUANTITY", "Min"],
                ["ACTIVITY_QUANTITY", "Max"],
            ],
            shape_unit="ACRES",
        )
    
        temp_footprint = os.path.join(scratch_workspace, f"Footprint_Report_{year}_{date_id}")
        
        select_11 = arcpy.analysis.Select(
            in_features=Dinner, 
            out_feature_class=temp_footprint
        )


        arcpy.management.CalculateField(
            in_table=select_11,
            field="Year_txt",
            expression="{}".format(year),
            expression_type="PYTHON3",
            code_block="",
            field_type="TEXT",
            enforce_domains="NO_ENFORCE_DOMAINS",
        )
        
        arcpy.AssignDomainToField_management(
            select_11, "PRIMARY_OWNERSHIP_GROUP", "D_PR_OWN_GR"
        )
        
        arcpy.AssignDomainToField_management(
            in_table=select_11, 
            field_name="COUNTY", 
            domain_name="D_CNTY"
        )
        
        arcpy.AssignDomainToField_management(
            in_table=select_11, 
            field_name="BROAD_VEGETATION_TYPE", 
            domain_name="D_BVT"
        )
        
        arcpy.AssignDomainToField_management(
            in_table=select_11, 
            field_name="REGION", 
            domain_name="D_TASKFORCE"
        )

        temp_footprint_pt = os.path.join(scratch_workspace, f"Footprint_pts{year}_{date_id}")


        # Process: Feature To Point (2) (Feature To Point) (management)
        # Footprint_Report_2021_2022_pts = os.path.join(workspace, "f_Report_Data", output_footprint)
        # if Footprint_Report_pts_20230720_lyrx and scratch_pt_3 and Value_3_:
        to_pt_1 = arcpy.management.FeatureToPoint(
            in_features=select_11,
            out_feature_class=temp_footprint_pt,
            point_location="INSIDE",
        )
        
        arcpy.AssignDomainToField_management(
            in_table=to_pt_1, 
            field_name="PRIMARY_OWNERSHIP_GROUP", 
            domain_name="D_PR_OWN_GR"
        )
        arcpy.AssignDomainToField_management(
            in_table=to_pt_1, 
            field_name="COUNTY", 
            domain_name="D_CNTY"
        )
        arcpy.AssignDomainToField_management(
            in_table=to_pt_1, 
            field_name="BROAD_VEGETATION_TYPE", 
            domain_name="D_BVT"
        )
        arcpy.AssignDomainToField_management(
            in_table=to_pt_1, 
            field_name="REGION", 
            domain_name="D_TASKFORCE"
        )
        
        arcpy.management.DeleteField(
            in_table=to_pt_1, 
            drop_field=["ORIG_FID"]
        )

        footprint_lst.append(select_11)
        footprint_pt_lst.append(to_pt_1)


    footprint_out_append = arcpy.management.CreateFeatureclass(
        out_path=os.path.dirname(output_footprint), 
        out_name=os.path.basename(output_footprint), 
        geometry_type="POLYGON", 
        template=footprint_lst[0]
        )
    
    footprint_out = arcpy.management.Append(
        inputs=footprint_lst,
        target=footprint_out_append,
        schema_type="NO_TEST",
    )

    footprint_pt_out_append = arcpy.management.CreateFeatureclass(
        out_path=os.path.dirname(output_footprint_pts), 
        out_name=os.path.basename(output_footprint_pts), 
        geometry_type="POINT", 
        template=footprint_pt_lst[0]
        )
    
    footprint_pt_out = arcpy.management.Append(
        inputs=footprint_pt_lst,
        target=footprint_pt_out_append,
        schema_type="NO_TEST",
    )
    """
    delete_scratch_files(
        gdb=scratch_workspace, 
        delete_fc="yes", 
        delete_table="yes", 
        delete_ds="yes"
    )
    """
    return footprint_out, footprint_pt_out

In [8]:
buffer_pts_1 = update_pt(input_fc_pt)
scratch_ln_3 = update_ln(input_fc_ln)
scratch_poly_1 = update_poly(input_fc_poly)

In [9]:


# APPEND point buffers, line buffers, and polygons
    
Footprints_Append = arcpy.management.CreateFeatureclass(
    out_path=scratch_workspace, 
    out_name="Footprints_Append", 
    geometry_type="POLYGON", 
    template=WFR_TF_Template
    )

append_1 = arcpy.management.Append(
    inputs=[
        buffer_pts_1,
        scratch_ln_3,
        scratch_poly_1,
    ],
    target=Footprints_Append,
    schema_type="NO_TEST",
)

In [10]:
get_footprint(append_1, output_footprint, output_footprint_pts)

Processing year:  2020
D:\WORK\wildfire\Interagency-Tracking-System\its\scratch.gdb\Footprints_Append_Where has 6095 records
D:\WORK\wildfire\Interagency-Tracking-System\its\scratch.gdb\Spaghetti_FeatureToPolygon has 28478 records
Processing year:  2021
D:\WORK\wildfire\Interagency-Tracking-System\its\scratch.gdb\Footprints_Append_Where has 9241 records
D:\WORK\wildfire\Interagency-Tracking-System\its\scratch.gdb\Spaghetti_FeatureToPolygon has 37969 records
Processing year:  2022
D:\WORK\wildfire\Interagency-Tracking-System\its\scratch.gdb\Footprints_Append_Where has 9469 records
D:\WORK\wildfire\Interagency-Tracking-System\its\scratch.gdb\Spaghetti_FeatureToPolygon has 38316 records
Processing year:  2023
D:\WORK\wildfire\Interagency-Tracking-System\its\scratch.gdb\Footprints_Append_Where has 11296 records
D:\WORK\wildfire\Interagency-Tracking-System\its\scratch.gdb\Spaghetti_FeatureToPolygon has 45935 records


(<Result 'D:\\WORK\\wildfire\\Interagency-Tracking-System\\its\\Interagency Tracking System.gdb\\f_Report_Data\\Footprint_Report_20240927'>,
 <Result 'D:\\WORK\\wildfire\\Interagency-Tracking-System\\its\\Interagency Tracking System.gdb\\f_Report_Data\\Footprint_pts_20240927'>)

In [11]:
delete_scratch_files(
    gdb=scratch_workspace, 
    delete_fc="yes", 
    delete_table="yes", 
    delete_ds="yes"
)

ExecuteError: ERROR 000464: Cannot get exclusive schema lock.  Either being edited or in use by another application or service.
Failed to execute (Delete).
